In [1]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
import gc
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import xgboost as xgb
from catboost import CatBoostClassifier

In [3]:
train_data = pd.read_csv(r'D:\学习\电影预测/train1217 16点30.csv')
train_label = pd.read_csv(r'D:\学习\qiye/train_label.csv')
train_data = train_data.merge(train_label, on='ID', how='left')

test_data = pd.read_csv(r'D:\学习\qiye/test_stage2_update_20200320.csv')
test_data['Label']=-1

D:\software\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
train_data = pd.read_csv(r'D:\学习\电影预测/train1217 16点30.csv')
def tf_idf(train, col, n_components=5):
    Tfidf_vect = TfidfVectorizer()
    tfidf_vec = Tfidf_vect.fit_transform(train[col])
    svd_enc = TruncatedSVD(n_components=n_components, n_iter=20, random_state=2020)
    vec_svd = svd_enc.fit_transform(tfidf_vec)
    vec_svd = pd.DataFrame(vec_svd)
    vec_svd.columns = ['SVD_{}_{}'.format(col, i) for i in range(n_components)]
    vec_svd = vec_svd.reset_index(drop= True)
    return vec_svd

In [6]:
def get_data_null_rate(train, test, cols=None):
    if cols!=None:
        train = train[cols]
        test = test[cols]
    data_null_rate = (train.isnull().sum()/len(train)).reset_index().rename(columns={0:'train'})
    data_null_rate = data_null_rate.merge((test.isnull().sum()/len(test)).reset_index().rename(columns={0:'test'}),
                                          on='index',how='inner')
    del train,test
    gc.collect()
    return data_null_rate
data_null_rate = get_data_null_rate(train_data.copy(),test_data.copy())
# 删除训练集和验证集中缺失率为1的 特征
rm_cols = list(data_null_rate[(data_null_rate['train']==1)|(data_null_rate['test']==1)]['index'])
for col in train_data.columns:
    if train_data[col].nunique() < 2:
        rm_cols.append(col)

train_data = train_data[[col for col in train_data.columns if col not in rm_cols]]
test_data = test_data[[col for col in test_data.columns if col not in rm_cols]]
all_data = train_data.append(test_data)

In [7]:
jingying_svd = tf_idf(all_data, '经营范围', n_components=5)
all_data = all_data.reset_index(drop= True)
all_data = pd.concat([all_data, jingying_svd], axis=1)

In [17]:
Tfidf_vect = TfidfVectorizer()
tfidf_vec = Tfidf_vect.fit_transform(all_data['经营范围'])

In [9]:
all_data.to_csv('山东alldata.csv')

In [20]:
train = pd.read_csv(r'D:\学习\电影预测/train.csv')


In [21]:
train_svd = tf_idf(train, 'crew', n_components=5)
train = train.reset_index(drop= True)
train = pd.concat([train, train_svd], axis=1)

In [22]:
train.to_csv('电影预测train.csv')